<a href="https://colab.research.google.com/github/c1t1zen1/PARROT_REPORT/blob/ART/FACE_MOSAIC_PARROT_REPORT_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PARROT REPORT MOSAIC FACES v1

In [ ]:
# @title Install Face Recognition & Connect G-Drive { vertical-output: true, display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')

!pip install face_recognition
!Nvidia-smi

In [ ]:
# @title Find All Faces Down Center of All Image { vertical-output: true, display-mode: "form" }
Folders = "['/content/drive/MyDrive/HTML_Kali/14_08_2023/tmp', '/content/drive/MyDrive/HTML_Kali/15_08_2023/tmp', '/content/drive/MyDrive/HTML_Kali/16_08_2023/tmp', '/content/drive/MyDrive/HTML_Kali/17_08_2023/tmp', '/content/drive/MyDrive/HTML_Kali/18_08_2023/tmp', '/content/drive/MyDrive/HTML_Kali/19_08_2023/tmp', '/content/drive/MyDrive/HTML_Kali/20_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/21_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/22_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/23_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/24_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/25_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/26_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/27_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/28_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/29_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/30_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/31_08_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/01_09_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/02_09_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/03_09_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/04_09_2023/Bimg/tmp', '/content/drive/MyDrive/HTML_Kali/05_09_2023/Bimg/tmp']" # @param {type:"string"}
# Import necessary libraries
import os
import face_recognition

# Ensure CUDA GPU is being used by Dlib (backend for face_recognition)
import dlib
if dlib.cuda.get_num_devices() == 0:
    raise ValueError("No CUDA GPUs detected!")
dlib.DLIB_USE_CUDA = True

# Path to the directory containing images on Google Drive
image_directories = Folders

# List of valid image extensions
valid_image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

# Threshold (in pixels) for determining if a face is horizontally centered
center_threshold = 100

# Define a minimum face size threshold (e.g., 3000 square pixels)
min_face_size = 3000

# List to store paths of images containing horizontally centered human faces
centered_face_images = []

def process_directory(directory):
    """Recursively process a directory and its subdirectories."""
    for item_name in os.listdir(directory):
        item_path = os.path.join(directory, item_name)

        # Check if the item is a file
        if os.path.isfile(item_path) and os.path.splitext(item_path)[1].lower() in valid_image_extensions:
            # Load the image using face_recognition
            image = face_recognition.load_image_file(item_path)

            # Find all face locations in the image using the CNN model
            face_locations = face_recognition.face_locations(image, model="cnn")

            # Calculate the horizontal center of the image using its dimensions
            image_horizontal_center = image.shape[1] // 2

            for face_location in face_locations:
                top, right, bottom, left = face_location

                # Calculate the area of the detected face bounding box
                face_area = (right - left) * (bottom - top)

                # If the face is smaller than the minimum size, skip to the next face
                if face_area < min_face_size:
                    continue

                # Calculate the horizontal center of the detected face bounding box
                face_horizontal_center = (left + right) // 2

                # Compute the difference between the image's horizontal center and face's horizontal center
                horizontal_difference = abs(face_horizontal_center - image_horizontal_center)

                # If the difference is below the threshold, consider the face as horizontally centered
                if horizontal_difference < center_threshold:
                    print(f"Found horizontally centered foreground face in {item_path}")
                    centered_face_images.append(item_path)
                    break

        # If the item is a directory, process it recursively
        elif os.path.isdir(item_path):
            process_directory(item_path)

# Start processing from the root directory
for directory in image_directories:
    process_directory(directory)

# Specify the path to save the output text file on Google Drive
output_file_path = '/content/drive/MyDrive/MosaicFaces.txt'

# Save the list of images with horizontally centered human faces to the text file
with open(output_file_path, 'w') as file:
    for image_path in centered_face_images:
        file.write(image_path + '\n')

print(f"List of images with horizontally centered human faces saved to {output_file_path}")


In [ ]:
# @title Copy MosaicFaces To Folder { vertical-output: true, display-mode: "form" }
import shutil
from datetime import datetime

# Path to the directory where centered face images will be copied
output_directory = '/content/drive/MyDrive/MosaicFaces'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Load the list of images with centered human faces from the text file
with open(output_file_path, 'r') as file:
    centered_face_images = file.readlines()

# Copy the images to the output directory with unique names based on timestamps
for image_path in centered_face_images:
    image_path = image_path.strip()  # Remove any trailing whitespace
    timestamp = datetime.now().strftime('%Y%m%d%H%M%S%f')
    new_filename = f"centered_face_{timestamp}{os.path.splitext(image_path)[1]}"
    shutil.copy(image_path, os.path.join(output_directory, new_filename))

print(f"Centered face images copied to {output_directory}")


In [ ]:
# @title Delete Duplicates { vertical-output: true, display-mode: "form" }
# Remove Duplicates
import os
import hashlib

def compute_md5(image_path):
    """Compute the MD5 hash of an image."""
    with open(image_path, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

# Define the directory to scan
image_directory = '/content/drive/MyDrive/MosaicFaces'

# Dictionary to store image hashes
image_hashes = {}

# List to store paths of duplicate images
duplicates = []

for image_path in os.listdir(image_directory):
    full_path = os.path.join(image_directory, image_path)

    # Compute the MD5 hash for the image
    image_hash = compute_md5(full_path)

    # If the hash already exists in the dictionary, it's a duplicate
    if image_hash in image_hashes:
        duplicates.append(full_path)
        os.remove(full_path)  # Delete the duplicate image
    else:
        image_hashes[image_hash] = full_path

# Print out the deleted duplicates
if duplicates:
    print("Deleted duplicate images:")
    for dup in duplicates:
        print(dup)
else:
    print("No duplicate images found.")